<a href="https://colab.research.google.com/github/wilberquito/AMLProject/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project: Image Classification
## Advanced Machine Learning


> Wilber E. Bermeo Quito 
>
> Judit Quintana Massana
>
> April 2023

In [1]:
import zipfile
from pathlib import Path
import torch
import matplotlib.pyplot as plt

In [2]:
try:
    import google.colab
    IN_COLAB = True
    ! pip install torchvision
    ! pip install torchinfo
    
    from google.colab import drive
    drive.mount('/content/drive')

    !rm -rf test
    data_path = Path('/content/drive/MyDrive/AML/test_unlabelled.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall("test")

    !rm -rf modular
    data_path = Path('/content/drive/MyDrive/AML/modular.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall(".")

    !rm -rf trained
    !cp -rf /content/drive/MyDrive/AML/trained ./trained
except:
    IN_COLAB = False

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
from modular.evaluate import val_step
import modular.models as models
from modular.datasets import TestDataset
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

data_path = "./test/test_unlabelled"
out_dim = 10
n_test = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
models = {
    Path('./trained/AMLMAXVIT_T.pth'): models.AMLMAXVIT_T(out_dim)
}

Path('./predict').mkdir(exist_ok=True, parents=True)

for p, m in models.items():
  data = torch.load(p, map_location=device)
  m.load_state_dict(data['model'])
  m = m.to(device)
  transform = m.transforms
  dataset = TestDataset(data_path, transform)
  dataloader = DataLoader(dataset, batch_size=256, shuffle=False)
  _, probs, labels = val_step(m, dataloader, 
                                    device, out_dim, n_test)
  
  predict_name = Path('./predict') / Path(p.name.replace('.pth', '.csv'))
  df = pd.DataFrame(probs)
  df.index = dataset.file_names
  df.to_csv(predict_name)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaxVit_T_Weights.IMAGENET1K_V1`. You can also use `weights=MaxVit_T_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 40/40 [05:29<00:00,  8.23s/it]


In [6]:
probs.shape, labels.shape

((10000, 10), (10000,))

In [ ]:
dataset.file_names

In [16]:
p.name.replace('.pth', '.csv')

'AMLMAXVIT_T.csv'

In [11]:
import pandas as pd
df = pd.DataFrame(probs)
df.index = dataset.file_names
df

,0,1,2,3,4,5,6,7,8,9
0000.jpg,0.567783,0.030058,0.024534,0.288866,0.030749,0.040912,0.002247,0.002106,0.008411,0.004334
0001.jpg,0.006919,0.000088,0.301789,0.059635,0.508114,0.074919,0.033786,0.005795,0.008264,0.000691
0002.jpg,0.000558,0.000058,0.000794,0.827720,0.018211,0.108021,0.040053,0.004256,0.000033,0.000298
0003.jpg,0.300947,0.011649,0.097755,0.317119,0.181971,0.060811,0.017334,0.008754,0.001025,0.002635
0004.jpg,0.001281,0.093503,0.003096,0.541092,0.003325,0.001667,0.005767,0.003450,0.020189,0.326629
...,...,...,...,...,...,...,...,...,...,...
9995.jpg,0.007706,0.000102,0.009481,0.105587,0.031172,0.292029,0.001790,0.047691,0.481850,0.022592
9996.jpg,0.138847,0.842912,0.000186,0.000272,0.000173,0.000179,0.000141,0.000038,0.005366,0.011886
9997.jpg,0.009696,0.964427,0.002755,0.002297,0.000281,0.001827,0.000816,0.000177,0.002523,0.015201
9998.jpg,0.001570,0.000142,0.062728,0.527038,0.096917,0.234054,0.026666,0.049989,0.000330,0.000567
